#### Environment initialization

In [ ]:
try:
    import os
    from kaggle_secrets import UserSecretsClient  # type: ignore

    secrets_client = UserSecretsClient()
    os.environ['GITHUB_TOKEN'] = secrets_client.get_secret('GITHUB_TOKEN')
    os.environ['WANDB_TOKEN'] = secrets_client.get_secret('WANDB_TOKEN')

    os.environ['__KGLTBX_INSTALL_FROM_GITHUB'] = '1'
    os.environ['__KGLTBX_ENVIRONMENT'] = 'kaggle'
except Exception:
    print('Kaggle initialization failed, probably not running on Kaggle...')

In [ ]:
try:
    import os
    from google.colab import drive  # type: ignore

    drive.mount('/content/drive')

    with open('/content/drive/MyDrive/credentials/.env') as f:
        for line in f:
            line = line.strip()
            k, v, *_ = line.split('=')
            os.environ[k] = v

    os.environ['__KGLTBX_INSTALL_FROM_GITHUB'] = '1'
    os.environ['__KGLTBX_ENVIRONMENT'] = 'colab'
except Exception:
    print('Colab initialization failed, probably not running on Colab...')


#### Requirements

In [ ]:
%%writefile requirements.txt

iterative-stratification==0.1.7

git+https://${GITHUB_TOKEN}@github.com/andrei-papou/kaggle-toolbox.git@main#egg=kaggle_toolbox[remote,wandb]

In [ ]:
# Install requirements only when running on Kaggle.
!if [ "$__KGLTBX_INSTALL_FROM_GITHUB" == "1" ]; then pip install -r requirements.txt; fi
!rm requirements.txt

#### Imports

In [ ]:
import itertools
import math
import os
import typing as t
from pathlib import Path

import pandas as pd
import torch
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from kaggle_toolbox import device
from kaggle_toolbox.data import DatasetItem, DatasetItemCollator
from kaggle_toolbox.device import CUDADevice
from kaggle_toolbox.environment import Environment
from kaggle_toolbox.features.transform import contiguous_to_categorical
from kaggle_toolbox.iter import FixedSubsetIterPlannerBuilder, FracSubsetSize
from kaggle_toolbox.layers import SqueezeDim
from kaggle_toolbox.logging.base import Logger
from kaggle_toolbox.logging.stdout import StdOutLogger
from kaggle_toolbox.logging.wandb import WAndBLogger
from kaggle_toolbox.loss.regression import MSELoss
from kaggle_toolbox.lr_scheduling import create_cosine_scheduler_with_warmup
from kaggle_toolbox.metrics.regression import MSEMetric
from kaggle_toolbox.nlp.transformer import Backbone, Model as TransformerModel, StandardModel, \
    MeanPooler, AttentionHeadPooler, TakeNthSqueezer, ConcatSqueezer, create_nakama_optimizer, get_tokenizer_for_backbone, \
    Tokenizer, TokenizerResult, TokenizerResultCollator, seed_everything, standard_init_linear, \
    standard_init_layer_norm
from kaggle_toolbox.nlp.transformer.model import MultiStagedDropout
from kaggle_toolbox.oof import OOFPredDict
from kaggle_toolbox.path import format_path
from kaggle_toolbox.progress import NotebookProgressBar
from kaggle_toolbox.trainer import StandardIterationTrainer, FullCycleTrainer, train_kfold_model
from kaggle_toolbox.typing import DynamicDict, filter_maybe_list
from kaggle_toolbox.validation import analyze_val_strategy, build_fold_result_df
from torch.optim import Optimizer, AdamW
from torch.utils.data import Dataset as TorchDataset
from transformers.data.data_collator import DataCollatorWithPadding
from transformers.utils.generic import PaddingStrategy
from transformers.utils.logging import set_verbosity_error as set_transformers_verbosity_error


NotebookProgressBar.attach_to_pandas()
set_transformers_verbosity_error()

#### Dataset

In [ ]:
class Dataset(TorchDataset[DatasetItem[TokenizerResult]]):

    def __init__(
            self,
            df: pd.DataFrame,
            tokenizer: Tokenizer,
            max_len: int,
            target: str):
        self._df = df.copy().reset_index(drop=True)
        self._tokenizer = tokenizer
        self._max_len = max_len
        self._target = target

    def _get_tokenizer_input(self, row: DynamicDict) -> str:
        (
            full_text,
         ) = (
            row.get_typed_or_raise('full_text', str),
         )

        return full_text

    def sort_by_tokenizer_input_len(self):
        self._df['_tok_input_len'] = self._df.progress_apply(
            lambda row: self._get_tokenizer_input(DynamicDict(t.cast(t.Dict[str, t.Any], row))), axis=1)
        self._df = self._df.sort_values('_tok_input_len')

    def __len__(self) -> int:
        return len(self._df)

    def __getitem__(self, idx: int) -> DatasetItem[TokenizerResult]:
        row = self._df.iloc[idx]

        tokenizer_input = self._get_tokenizer_input(DynamicDict(t.cast(t.Dict[str, t.Any], row)))
        id = str(row['text_id'])

        tokenizer_result = self._tokenizer.tokenize(
            tokenizer_input, max_len=self._max_len)
        target_tensor = torch.tensor(float(row[self._target]), dtype=torch.float32)

        return DatasetItem(
            id=[id],
            x=tokenizer_result,
            y=target_tensor)

#### Parameters

In [ ]:
TARGET_LIST = [
    'cohesion',
    'syntax',
    'vocabulary',
    'phraseology',
    'grammar',
    'conventions',
]
TARGET = TARGET_LIST[2]
ENVIRONMENT = os.getenv('__KGLTBX_ENVIRONMENT', 'laptop')
_env = _env = Environment(ENVIRONMENT)

IS_KAGGLE = 'KAGGLE_URL_BASE' in os.environ
IS_PRERUN = False
SEED = 42
NUM_FOLDS = 5
FOLD_LIST = [0, 1, 2] if not IS_PRERUN else [0]
DEVICE = CUDADevice()
BACKBONE = 'microsoft/deberta-v3-base'
MAX_LEN = 1024
ENCODER_LR = 1e-5
DECODER_LR = 1e-4
BATCH_SIZE = _env.param(kaggle=4, colab=4, laptop=1)
ACCUMULATE_GRADIENT_STEPS = _env.param(kaggle=2, colab=2, laptop=8)
NUM_EPOCHS = 3
VAL_FREQ = 0.25
NUM_WORKERS = _env.param(kaggle=2, colab=2, laptop=4)

ROOT_DIR = _env.param(
    kaggle=Path('/kaggle'),
    colab=Path('/content/drive/MyDrive'),
    laptop=Path('/kaggle'))
DATA_DIR = _env.param(
    kaggle=ROOT_DIR / 'input',
    colab=ROOT_DIR / 'data',
    laptop=ROOT_DIR / 'data')
FP_ELL_DATASET_DIR = _env.param(
    kaggle=DATA_DIR / 'feedback-prize-english-language-learning',
    colab=DATA_DIR / 'fp-ell',
    laptop=DATA_DIR / 'fp-ell')
MODEL_DIR = _env.param(
    kaggle=ROOT_DIR / 'working',
    colab=ROOT_DIR / 'models/fp-ell',
    laptop=ROOT_DIR / 'models')
OOF_DIR = _env.param(
    kaggle=ROOT_DIR / 'working',
    colab=ROOT_DIR / 'oof/fp-ell',
    laptop=ROOT_DIR / 'oof')

RUN_ID = f'{TARGET}-v1-layer_norm-ep_3-valfreq_0p25-std_init'
MODEL_PATH_TEMPLATE = MODEL_DIR / f'{RUN_ID}-fold_{{fold}}.pt'
OOF_PATH = OOF_DIR / f'{RUN_ID}.csv'

In [ ]:
print(f'GPU model: {DEVICE.get_name()}')

#### Pinning the seed

In [ ]:
seed_everything(seed=SEED)

#### Data loading

In [ ]:
def _read_data(dataset_dir_path: Path, target_list: t.List[str], num_folds: int, seed: int) -> pd.DataFrame:
    all_df = pd.read_csv(dataset_dir_path / 'train.csv')
    target_arr = contiguous_to_categorical(all_df[target_list].values)

    mskf = MultilabelStratifiedKFold(n_splits=num_folds, shuffle=True, random_state=seed)
    for fold_, (_, v_) in enumerate(mskf.split(X=all_df, y=target_arr)):
        all_df.loc[v_, 'fold'] = fold_

    return all_df

all_df = _read_data(
    dataset_dir_path=FP_ELL_DATASET_DIR,
    target_list=TARGET_LIST,
    num_folds=NUM_FOLDS,
    seed=SEED)

analyze_val_strategy(all_df, target_list=TARGET_LIST, num_folds=NUM_FOLDS)

#### Entrypoint

In [ ]:
def _train_model(fold: int) -> t.Tuple[float, OOFPredDict]:
    backbone = Backbone.from_huggingface_checkpoint(BACKBONE, zero_out_dropout=True)
    tokenizer = get_tokenizer_for_backbone(backbone=BACKBONE, padding_strategy=PaddingStrategy.DO_NOT_PAD)
    model: StandardModel[TokenizerResult] = StandardModel(
        backbone=backbone,
        squeezer=TakeNthSqueezer(),
        # pooler=AttentionHeadPooler(backbone.out_dim_size),
        pooler=MeanPooler(),
        dnn=torch.nn.Sequential(
            # torch.nn.LayerNorm(backbone.out_dim_size),
            # torch.nn.Linear(backbone.out_dim_size, 1),
            standard_init_layer_norm(torch.nn.LayerNorm(backbone.out_dim_size)),
            standard_init_linear(torch.nn.Linear(backbone.out_dim_size, 1)),
            SqueezeDim(),
        ))
    optimizer = create_nakama_optimizer(
        model=model,
        encoder_lr=ENCODER_LR,
        decoder_lr=DECODER_LR)

    train_df, valid_df = all_df[all_df['fold'] != fold], all_df[all_df['fold'] == fold]

    train_dataset = Dataset(
        df=train_df,
        tokenizer=tokenizer,
        max_len=MAX_LEN,
        target=TARGET)
    valid_dataset = Dataset(
        df=valid_df,
        tokenizer=tokenizer,
        max_len=MAX_LEN,
        target=TARGET)
    valid_dataset.sort_by_tokenizer_input_len()

    num_training_steps = (len(train_dataset) * NUM_EPOCHS) // (BATCH_SIZE * ACCUMULATE_GRADIENT_STEPS)

    trainer: FullCycleTrainer[TokenizerResult] = FullCycleTrainer(
        iteration_trainer=StandardIterationTrainer(
            model=model,
            criterion=MSELoss(),
            optimizer=optimizer,
            scheduler=create_cosine_scheduler_with_warmup(
                optimizer=optimizer,
                num_training_steps=num_training_steps,
                warmup_steps_ratio=0.0,
                num_cycles=0.5),
            pred_quality_metric_list=[
                MSEMetric(squared=False),
            ],
            device=DEVICE,
            accumulate_gradient_steps=ACCUMULATE_GRADIENT_STEPS,
            progress_bar=NotebookProgressBar()),
        train_iter_planner_builder=FixedSubsetIterPlannerBuilder(FracSubsetSize(VAL_FREQ)),
        batch_size=BATCH_SIZE,
        collator=DatasetItemCollator(
            id_collate_fn=lambda x: sum(x, []),
            x_collate_fn=TokenizerResultCollator(DataCollatorWithPadding(tokenizer.tokenizer))),
        num_epochs=NUM_EPOCHS,
        num_workers=NUM_WORKERS,
        model_comparison_metric=MSEMetric.valid_name(),
        model_comparison_metric_criteria=MSEMetric.criteria,
        save_model_to_path=format_path(MODEL_PATH_TEMPLATE, fold=str(fold)),
        logger_list=filter_maybe_list([
            StdOutLogger() if not IS_PRERUN else None,
            WAndBLogger(
                user_name='andrei-papou',
                project='fp-ell',
                run_id=RUN_ID,
                metric_prefix=f'f{fold}'
            ) if not IS_PRERUN else None,
        ]))

    return trainer.do_full_cycle(train_dataset, valid_dataset)

score_list, oof_pred_dict = train_kfold_model(
    train_model_fn=_train_model,
    fold_list=FOLD_LIST)
oof_pred_dict.save_to_csv(
    OOF_PATH,
    score_col_name_list=[f'{TARGET}_score'])
build_fold_result_df(fold_list=FOLD_LIST, score_list=score_list)

#### Environment shutdown

In [ ]:
if ENVIRONMENT == 'colab':
    from google.colab import runtime  # type: ignore

    runtime.unassign()